# Ý nghĩa các cột
    Column	                        Description
    POSTED_BY	                Category marking who has listed the property        Danh sách  người post
    UNDER_CONSTRUCTION	        Under Construction or Not                           Đang xây dựng hay đã haonf thiện
    RERA	                        Rera approved or Not                                Rera được phép hay không
    BHK_NO	                        Number of Rooms                                     Số lượng phòng
    BHKORRK	                        Type of property                                    Loại tài sản
    SQUARE_FT	                Total area of the house in square feet              Tổng diện tích ngôi nhà tính bằng feet
    READYTOMOVE	                Category marking Ready to move or Not               Sẵn sàng di chuuyển hay chưa
    RESALE	                        Category marking Resale or not                      Đã được mua bán lại hay chưa
    ADDRESS	                        Address of the property                             Địa chỉ
    LONGITUDE	                Longitude of the property                           Chiều dài
    LATITUDE	                Latitude of the property                            Chiều rộng

In [21]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
import pypyodbc as odbc
import squarify
import pyodbc
import requests
from io import StringIO
from sklearn.linear_model import LinearRegression
print('Completed import lib')

Completed import lib


In [22]:
# Với file raw từ github cần thao tác như sau
url = 'https://raw.githubusercontent.com/DK-Hieu/House_price_prediction/main/ver_2_data_long/train.csv'
req = requests.get(url)
data = StringIO(req.text)
data_train = pd.read_csv(data)

url = 'https://raw.githubusercontent.com/DK-Hieu/House_price_prediction/main/ver_2_data_long/test.csv'
req = requests.get(url)
data = StringIO(req.text)
data_test = pd.read_csv(data)

#data_train.drop(columns=['ADDRESS','LONGITUDE','LATITUDE'], axis=1, inplace=True)
#data_test.drop(columns=['ADDRESS','LONGITUDE','LATITUDE'], axis=1, inplace=True)

In [23]:
data_train

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS)
0,Owner,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0
1,Dealer,0,0,2,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0
2,Owner,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0
3,Owner,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5
4,Dealer,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5
...,...,...,...,...,...,...,...,...,...,...,...,...
29446,Owner,0,0,3,BHK,2500.000000,1,1,"Shamshabad Road,Agra",27.140626,78.043277,45.0
29447,Owner,0,0,2,BHK,769.230769,1,1,"E3-108, Lake View Recidency,,Vapi",39.945409,-86.150721,16.0
29448,Dealer,0,0,2,BHK,1022.641509,1,1,"Ajmer Road,Jaipur",26.928785,75.828002,27.1
29449,Owner,0,0,2,BHK,927.079009,1,1,"Sholinganallur,Chennai",12.900150,80.227910,67.0


In [24]:
data_train.isnull().sum()

POSTED_BY                0
UNDER_CONSTRUCTION       0
RERA                     0
BHK_NO.                  0
BHK_OR_RK                0
SQUARE_FT                0
READY_TO_MOVE            0
RESALE                   0
ADDRESS                  0
LONGITUDE                0
LATITUDE                 0
TARGET(PRICE_IN_LACS)    0
dtype: int64

In [25]:
data_test

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE
0,Owner,0,0,1,BHK,545.171340,1,1,"Kamrej,Surat",21.262000,73.047700
1,Dealer,1,1,2,BHK,800.000000,0,0,"Panvel,Lalitpur",18.966114,73.148278
2,Dealer,0,0,2,BHK,1257.096513,1,1,"New Town,Kolkata",22.592200,88.484911
3,Dealer,0,0,3,BHK,1400.329489,1,1,"Kalwar Road,Jaipur",26.988300,75.584600
4,Owner,0,0,1,BHK,430.477830,1,1,"Mai Mandir,Nadiad",22.700000,72.870000
...,...,...,...,...,...,...,...,...,...,...,...
68715,Dealer,0,1,2,BHK,856.555505,1,1,"Thane West,Maharashtra",19.180000,72.963330
68716,Dealer,0,1,3,BHK,2304.147465,1,1,"Sector-66A Mohali,Mohali",30.661104,76.746082
68717,Dealer,1,1,1,BHK,33362.792750,0,0,"Balkum,Maharashtra",19.222101,72.988231
68718,Dealer,0,0,2,BHK,1173.708920,1,1,"Hadapsar,Pune",18.496670,73.941670


In [26]:
data_test.isnull().sum()

POSTED_BY             0
UNDER_CONSTRUCTION    0
RERA                  0
BHK_NO.               0
BHK_OR_RK             0
SQUARE_FT             0
READY_TO_MOVE         0
RESALE                0
ADDRESS               0
LONGITUDE             0
LATITUDE              0
dtype: int64

## EDA

### Chuyển đổi các dữ liệu dạng category thành số

In [30]:
data_train.corr()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,SQUARE_FT,READY_TO_MOVE,RESALE,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS)
POSTED_BY,1.000000,0.263473,0.285316,0.072466,-0.005415,-0.263473,-0.332280,0.049164,-0.025425,0.093328
UNDER_CONSTRUCTION,0.263473,1.000000,0.363826,-0.040712,-0.004204,-1.000000,-0.347405,0.006440,-0.000381,0.055399
RERA,0.285316,0.363826,1.000000,0.009547,-0.006229,-0.363826,-0.270351,0.104976,-0.065106,0.067636
BHK_NO.,0.072466,-0.040712,0.009547,1.000000,0.005303,0.040712,0.014581,0.068730,0.046930,0.112283
SQUARE_FT,-0.005415,-0.004204,-0.006229,0.005303,1.000000,0.004204,0.001732,-0.012591,0.000803,0.402685
READY_TO_MOVE,-0.263473,-1.000000,-0.363826,0.040712,0.004204,1.000000,0.347405,-0.006440,0.000381,-0.055399
RESALE,-0.332280,-0.347405,-0.270351,0.014581,0.001732,0.347405,1.000000,0.024038,0.014844,-0.207378
LONGITUDE,0.049164,0.006440,0.104976,0.068730,-0.012591,-0.006440,0.024038,1.000000,-0.155062,-0.031112
LATITUDE,-0.025425,-0.000381,-0.065106,0.046930,0.000803,0.000381,0.014844,-0.155062,1.000000,-0.017254
TARGET(PRICE_IN_LACS),0.093328,0.055399,0.067636,0.112283,0.402685,-0.055399,-0.207378,-0.031112,-0.017254,1.000000


In [27]:
#POST_BY
data_train['POSTED_BY'].unique()

array(['Owner', 'Dealer', 'Builder'], dtype=object)

In [28]:
data_train['BHK_OR_RK'].unique()

array(['BHK', 'RK'], dtype=object)

In [29]:
data_train.replace({'Owner':0,'Dealer':1,'Builder':2}, inplace=True)